### Defining the function to process txt datasets

In [1]:
import pandas as pd

def load_and_process_data(filename):
    try:
        # Read the data file
        df = pd.read_csv(filename, delim_whitespace=True, header=None)

        # The first column is the class label
        df.columns = ['class'] + ['feature_'+str(i) for i in range(1, len(df.columns))]

        # Move the 'class' column to the end
        df = df[[col for col in df.columns if col != 'class'] + ['class']]
        return df

    except FileNotFoundError:
        print(f"The file {filename} does not exist.")
    except pd.errors.ParserError:
        print(f"There was a problem parsing the file {filename}. Check if the file format is correct.")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

### The nearest neighbor function

In [2]:
from sklearn.model_selection import LeaveOneOut
from scipy.spatial import distance
import numpy as np

def nearest_neighbor(df, features):
    loo = LeaveOneOut()
    correct_classifications = 0
    total_classifications = 0

    # Convert features and class columns to numpy arrays for efficient calculation
    feature_data = df[features].values
    class_data = df['class'].values

    for train_index, test_index in loo.split(feature_data):
        X_train, X_test = feature_data[train_index], feature_data[test_index]
        y_train, y_test = class_data[train_index], class_data[test_index]

        # Calculate Euclidean distances and find the nearest neighbor
        distances = np.sqrt(((X_train - X_test)**2).sum(axis=1))
        nearest_neighbor_index = np.argmin(distances)

        # Classify the test data point based on its nearest neighbor
        predicted_class = y_train[nearest_neighbor_index]

        if predicted_class == y_test[0]:
            correct_classifications += 1

        total_classifications += 1

    # Calculate and return the accuracy
    accuracy = correct_classifications / total_classifications
    return accuracy


### The forward selection function

In [3]:
def forward_selection(df):
    current_set_of_features = []  # Start with an empty set
    best_overall_accuracy = 0
    best_overall_features = []
    print("\nBeginning search.")
    for i in range(len(df.columns) - 1):  # The '-1' is to exclude the class column
        feature_to_add_at_this_level = None
        best_so_far_accuracy = 0

        for feature in df.columns[:-1]:  # The '[:-1]' is to exclude the class column
            if feature not in current_set_of_features:
                accuracy = nearest_neighbor(df, current_set_of_features + [feature])

                print(f"\tUsing feature(s) {current_set_of_features + [feature]} accuracy is {accuracy * 100:.1f}%")

                if accuracy > best_so_far_accuracy:
                    best_so_far_accuracy = accuracy
                    feature_to_add_at_this_level = feature

        current_set_of_features.append(feature_to_add_at_this_level)

        print(f"\nFeature set {current_set_of_features} was best, accuracy is {best_so_far_accuracy * 100:.1f}%\n")

        if best_so_far_accuracy > best_overall_accuracy:
            best_overall_accuracy = best_so_far_accuracy
            best_overall_features = current_set_of_features.copy()
        elif feature_to_add_at_this_level is not None:
            print("\n(Warning, Accuracy has decreased! Continuing search in case of local maxima)")
        else:
            print("\nCannot improve by adding any features. Stopping.")
            break

    print(f"Finished search!! The best feature subset is {best_overall_features}, which has an accuracy of {best_overall_accuracy * 100:.1f}%")
    return best_overall_features, best_overall_accuracy


### The backward elimination function

In [4]:
def backward_elimination(df):
    current_set_of_features = df.columns[:-1].tolist()  # Start with all features
    best_overall_accuracy = 0
    best_overall_features = []

    print("Beginning search.\n")

    for i in range(len(df.columns) - 1):  # The '-1' is to exclude the class column
        feature_to_remove_at_this_level = None
        best_so_far_accuracy = 0

        for feature in current_set_of_features:
            test_features = current_set_of_features.copy()
            test_features.remove(feature)
            accuracy = nearest_neighbor(df, test_features)

            print(f"\tUsing feature(s) {test_features} accuracy is {accuracy * 100:.1f}%")

            if accuracy > best_so_far_accuracy:
                best_so_far_accuracy = accuracy
                feature_to_remove_at_this_level = feature

        current_set_of_features.remove(feature_to_remove_at_this_level)

        print(f"\nFeature set {current_set_of_features} was best, accuracy is {best_so_far_accuracy * 100:.1f}%\n")

        if best_so_far_accuracy > best_overall_accuracy:
            best_overall_accuracy = best_so_far_accuracy
            best_overall_features = current_set_of_features.copy()
        elif feature_to_remove_at_this_level is not None:
            print("\n(Warning, Accuracy has decreased! Continuing search in case of local maxima)")
        else:
            print("\nCannot improve by removing any features. Stopping.")
            break

    print(f"Finished search!! The best feature subset is {best_overall_features}, which has an accuracy of {best_overall_accuracy * 100:.1f}%")
    return best_overall_features, best_overall_accuracy


### Forward selection on the small dataset
As the day of the month for the birthday of the youngest member is 4th May, we are going to use CS170_small_Data__4.txt

In [8]:
filename = "CS170_small_Data__4.txt"
df = load_and_process_data(filename)
num_features = len(df.columns) - 1
num_instances = len(df)
print(f"\nThis dataset has {num_features} features (not including the class attribute), with {num_instances} instances.")
forward_selection(df)


This dataset has 10 features (not including the class attribute), with 1000 instances.

Beginning search.
	Using feature(s) ['feature_1'] accuracy is 72.2%
	Using feature(s) ['feature_2'] accuracy is 71.0%
	Using feature(s) ['feature_3'] accuracy is 69.5%
	Using feature(s) ['feature_4'] accuracy is 73.4%
	Using feature(s) ['feature_5'] accuracy is 83.2%
	Using feature(s) ['feature_6'] accuracy is 68.7%
	Using feature(s) ['feature_7'] accuracy is 70.6%
	Using feature(s) ['feature_8'] accuracy is 69.8%
	Using feature(s) ['feature_9'] accuracy is 70.1%
	Using feature(s) ['feature_10'] accuracy is 75.6%

Feature set ['feature_5'] was best, accuracy is 83.2%

	Using feature(s) ['feature_5', 'feature_1'] accuracy is 80.4%
	Using feature(s) ['feature_5', 'feature_2'] accuracy is 82.3%
	Using feature(s) ['feature_5', 'feature_3'] accuracy is 82.7%
	Using feature(s) ['feature_5', 'feature_4'] accuracy is 82.6%
	Using feature(s) ['feature_5', 'feature_6'] accuracy is 84.0%
	Using feature(s) ['f

(['feature_5', 'feature_10'], 0.96)

### Backward elimination on the small dataset

In [9]:
filename = "CS170_small_Data__4.txt"
df = load_and_process_data(filename)
num_features = len(df.columns) - 1
num_instances = len(df)
print(f"\nThis dataset has {num_features} features (not including the class attribute), with {num_instances} instances.")
backward_elimination(df)


This dataset has 10 features (not including the class attribute), with 1000 instances.
Beginning search.

	Using feature(s) ['feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10'] accuracy is 78.4%
	Using feature(s) ['feature_1', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10'] accuracy is 77.8%
	Using feature(s) ['feature_1', 'feature_2', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10'] accuracy is 77.1%
	Using feature(s) ['feature_1', 'feature_2', 'feature_3', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10'] accuracy is 75.8%
	Using feature(s) ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10'] accuracy is 71.5%
	Using feature(s) ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_7', 'feature_8', 'feature

(['feature_5', 'feature_10'], 0.96)

### Forward selection on the large dataset
As the day of the month for the birthday of the youngest member is 14th September, we are going to use CS170_large_Data__14.txt

In [10]:
filename = "CS170_large_Data__14.txt"
df = load_and_process_data(filename)
num_features = len(df.columns) - 1
num_instances = len(df)
print(f"\nThis dataset has {num_features} features (not including the class attribute), with {num_instances} instances.")
forward_selection(df)


This dataset has 20 features (not including the class attribute), with 2000 instances.

Beginning search.
	Using feature(s) ['feature_1'] accuracy is 69.2%
	Using feature(s) ['feature_2'] accuracy is 69.5%
	Using feature(s) ['feature_3'] accuracy is 84.0%
	Using feature(s) ['feature_4'] accuracy is 69.3%
	Using feature(s) ['feature_5'] accuracy is 71.1%
	Using feature(s) ['feature_6'] accuracy is 70.0%
	Using feature(s) ['feature_7'] accuracy is 71.4%
	Using feature(s) ['feature_8'] accuracy is 69.2%
	Using feature(s) ['feature_9'] accuracy is 73.1%
	Using feature(s) ['feature_10'] accuracy is 69.0%
	Using feature(s) ['feature_11'] accuracy is 69.2%
	Using feature(s) ['feature_12'] accuracy is 69.7%
	Using feature(s) ['feature_13'] accuracy is 69.9%
	Using feature(s) ['feature_14'] accuracy is 70.0%
	Using feature(s) ['feature_15'] accuracy is 69.0%
	Using feature(s) ['feature_16'] accuracy is 69.0%
	Using feature(s) ['feature_17'] accuracy is 69.4%
	Using feature(s) ['feature_18'] ac

(['feature_3', 'feature_9'], 0.9725)

### Backward elimination on the large dataset

In [11]:
filename = "CS170_large_Data__14.txt"
df = load_and_process_data(filename)
num_features = len(df.columns) - 1
num_instances = len(df)
print(f"\nThis dataset has {num_features} features (not including the class attribute), with {num_instances} instances.")
backward_elimination(df)


This dataset has 20 features (not including the class attribute), with 2000 instances.
Beginning search.

	Using feature(s) ['feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20'] accuracy is 71.2%
	Using feature(s) ['feature_1', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20'] accuracy is 71.9%
	Using feature(s) ['feature_1', 'feature_2', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20'] accuracy is 68.5%
	Using f

(['feature_3', 'feature_9'], 0.9725)

### Dealing with extra large dataset
The sum of the months of birth for both members is 14. Hence we are going to use CS170_XXXlarge_Data__14.txt for our extra large dataset. We can see in the following block of code that the dataset has 80 features and with all the features it takes 3.48 seconds to do leave-one-out cross validation. Hence its going to take approximately *(80*81*3.48)/2* seconds which comes around **3.13 hours**. To deal with this we are going to employ some strategies. 

In [13]:
filename = "CS170_XXXlarge_Data__14.txt"
df = load_and_process_data(filename)
num_features = len(df.columns) - 1
num_instances = len(df)
print(f"\nThis dataset has {num_features} features (not including the class attribute), with {num_instances} instances.")

import time
features = df.columns[:-1]  # all features except the class column

# Start the timer
start_time = time.time()

# Call the nearest_neighbor function
nearest_neighbor(df, features)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"The nearest neighbor function took {elapsed_time:.2f} seconds to process the given DataFrame and features.")




This dataset has 80 features (not including the class attribute), with 4000 instances.
The nearest neighbor function took 3.48 seconds to process the given DataFrame and features.


### Change to the nearest neighbor function
In the nearest neighbor function we are going to modify the control flow a little bit such that it also takes an extra parameter which signifies the maximum misclassifications allowed. If the number of misclassifications exceed this amount then we are simply going to exit the nearest neighbor function and return an accuracy of 0.0.

In [15]:
def alt_nn(df, features, min_misclassifications):
    loo = LeaveOneOut()
    misclassifications = 0
    total_classifications = 0

    feature_data = df[features].values
    class_data = df['class'].values

    for train_index, test_index in loo.split(feature_data):
        X_train, X_test = feature_data[train_index], feature_data[test_index]
        y_train, y_test = class_data[train_index], class_data[test_index]

        distances = np.sqrt(((X_train - X_test)**2).sum(axis=1))
        nearest_neighbor_index = np.argmin(distances)
        predicted_class = y_train[nearest_neighbor_index]

        if predicted_class != y_test[0]:
            misclassifications += 1
            if misclassifications >= min_misclassifications:
                return 0  # Early abandoning

        total_classifications += 1

    accuracy = (total_classifications - misclassifications) / total_classifications
    return accuracy

### Change to the forward selection algorithm
* We are going to change the forward selction algorithm. For each level of forward search first we are going to set the maximum misclassifications allowed to the total number of instances in the dataframe. Then we are going to update the maximum misclassifications allowed according to our best found accuracy and rest is taken care by the changed nearest neighbor function.

* Also we have defined a variable called **decrease_counter** to keep track of how many levels accuracy has decreased consecutively. If it exceeds 4 levels then we are going to halt the process and report the best accuraacy found till now. 

In [16]:
def fwd_selection(df):
    current_set_of_features = []  # Start with an empty set
    best_overall_accuracy = 0
    best_overall_features = []
    # Initialize to the total number of instances
    total_instances = len(df)
    min_misclassifications = total_instances 
    decrease_counter = 0  # Counter for decreases in accuracy

    print("\nBeginning search.")
    for i in range(len(df.columns) - 1):  # The '-1' is to exclude the class column
        feature_to_add_at_this_level = None
        best_so_far_accuracy = 0
        min_misclassifications = total_instances  # Reset min_misclassifications at the start of each level

        for feature in df.columns[:-1]:  # The '[:-1]' is to exclude the class column
            if feature not in current_set_of_features:
                accuracy = alt_nn(df, current_set_of_features + [feature], min_misclassifications)

                print(f"\tUsing feature(s) {current_set_of_features + [feature]} accuracy is {accuracy * 100:.1f}%")

                if accuracy > best_so_far_accuracy:
                    best_so_far_accuracy = accuracy
                    feature_to_add_at_this_level = feature
                    # Update min_misclassifications when we find a smaller value
                    min_misclassifications = total_instances - int(total_instances * best_so_far_accuracy)

        current_set_of_features.append(feature_to_add_at_this_level)

        print(f"\nFeature set {current_set_of_features} was best, accuracy is {best_so_far_accuracy * 100:.1f}%\n")

        if best_so_far_accuracy > best_overall_accuracy:
            best_overall_accuracy = best_so_far_accuracy
            best_overall_features = current_set_of_features.copy()
            decrease_counter = 0  # Reset decrease counter when accuracy increases
        elif feature_to_add_at_this_level is not None:
            print("\n(Warning, Accuracy has decreased! Continuing search in case of local maxima)")
            decrease_counter += 1  # Increment decrease counter when accuracy decreases
            if decrease_counter >= 4:  # Check if accuracy decreased 4 times in a row
                print("\nAccuracy has decreased for 4 consecutive levels. Halting the process.")
                break
        else:
            print("\nCannot improve by adding any features. Stopping.")
            break

    print(f"Finished search!! The best feature subset is {best_overall_features}, which has an accuracy of {best_overall_accuracy * 100:.1f}%\n")


### Running modified forward selection on the extra large dataset

In [17]:
filename = "CS170_XXXlarge_Data__14.txt"
df = load_and_process_data(filename)
fwd_selection(df)


Beginning search.
	Using feature(s) ['feature_1'] accuracy is 70.8%
	Using feature(s) ['feature_2'] accuracy is 0.0%
	Using feature(s) ['feature_3'] accuracy is 0.0%
	Using feature(s) ['feature_4'] accuracy is 0.0%
	Using feature(s) ['feature_5'] accuracy is 71.0%
	Using feature(s) ['feature_6'] accuracy is 0.0%
	Using feature(s) ['feature_7'] accuracy is 0.0%
	Using feature(s) ['feature_8'] accuracy is 0.0%
	Using feature(s) ['feature_9'] accuracy is 0.0%
	Using feature(s) ['feature_10'] accuracy is 0.0%
	Using feature(s) ['feature_11'] accuracy is 0.0%
	Using feature(s) ['feature_12'] accuracy is 0.0%
	Using feature(s) ['feature_13'] accuracy is 0.0%
	Using feature(s) ['feature_14'] accuracy is 0.0%
	Using feature(s) ['feature_15'] accuracy is 0.0%
	Using feature(s) ['feature_16'] accuracy is 0.0%
	Using feature(s) ['feature_17'] accuracy is 0.0%
	Using feature(s) ['feature_18'] accuracy is 0.0%
	Using feature(s) ['feature_19'] accuracy is 0.0%
	Using feature(s) ['feature_20'] accur

### Dealing with the real-world dataset
For our  real-world dataset we are going to use the Rice (Cammeo and Osmancik) dataset from Kaggle.The dataset has 7 attributes and all the attributes are continuous valued. The dataset has 2 Classes with values **Cammeo** and **Osmancik**. Also there are 3810 instances in the dataset.
* [Dataset page link](https://www.kaggle.com/datasets/muratkokludataset/rice-dataset-commeo-and-osmancik?resource=download)

In [34]:
import pandas as pd

# Load the data into a pandas DataFrame
df = pd.read_excel('Rice_Cammeo_Osmancik.xlsx')

# Print the head of the DataFrame
print(df['Class'].unique())
print(df.shape)
print(df.head())

['Cammeo' 'Osmancik']
(3810, 8)
    Area   Perimeter  Major_Axis_Length  Minor_Axis_Length  Eccentricity  \
0  15231  525.578979         229.749878          85.093788      0.928882   
1  14656  494.311005         206.020065          91.730972      0.895405   
2  14634  501.122009         214.106781          87.768288      0.912118   
3  13176  458.342987         193.337387          87.448395      0.891861   
4  14688  507.166992         211.743378          89.312454      0.906691   

   Convex_Area    Extent   Class  
0        15617  0.572896  Cammeo  
1        15072  0.615436  Cammeo  
2        14954  0.693259  Cammeo  
3        13368  0.640669  Cammeo  
4        15262  0.646024  Cammeo  


### Preprocessing the dataset
We see that the dataset has continuous values for all the attributes. So first we are going to **z-normalize** the attributes. Also the class column contains the **Cammeo** and **Osmancik** as class values. We are going to convert them to **1** and **2**. Also we are going to rename the **Class** column to **class** so that it works with our previously written code. 

In [35]:
from sklearn.preprocessing import StandardScaler

# Create an instance of StandardScaler
scaler = StandardScaler()

# Select continuous valued attributes
continuous_features = df.columns[df.columns != 'Class']

# Perform z-normalization on the continuous features
df[continuous_features] = scaler.fit_transform(df[continuous_features])

# Replace 'Cammeo' and 'Osmancik' in the 'Class' column with 1 and 2
df['Class'] = df['Class'].replace({'Cammeo': 1, 'Osmancik': 2})

# Renaming the 'Class' column to 'class'
df = df.rename(columns={'Class': 'class'})

# Check the preprocessed DataFrame
print(df.head())


       Area  Perimeter  Major_Axis_Length  Minor_Axis_Length  Eccentricity  \
0  1.479830   2.004354           2.348547          -0.212943      2.018337   
1  1.147870   1.125853           0.988390           0.945568      0.410018   
2  1.135169   1.317214           1.451908           0.253887      1.212956   
3  0.293436   0.115300           0.261439           0.198051      0.239751   
4  1.166345   1.487053           1.316442           0.523419      0.952221   

   Convex_Area    Extent  class  
0     1.499659 -1.152921      1  
1     1.192918 -0.602079      1  
2     1.126504  0.405611      1  
3     0.233857 -0.275351      1  
4     1.299855 -0.206013      1  


### Performing forward selection on the dataframe

In [36]:
forward_selection(df)


Beginning search.
	Using feature(s) ['Area'] accuracy is 79.9%
	Using feature(s) ['Perimeter'] accuracy is 87.9%
	Using feature(s) ['Major_Axis_Length'] accuracy is 88.7%
	Using feature(s) ['Minor_Axis_Length'] accuracy is 58.2%
	Using feature(s) ['Eccentricity'] accuracy is 69.0%
	Using feature(s) ['Convex_Area'] accuracy is 82.3%
	Using feature(s) ['Extent'] accuracy is 55.9%

Feature set ['Major_Axis_Length'] was best, accuracy is 88.7%

	Using feature(s) ['Major_Axis_Length', 'Area'] accuracy is 87.8%
	Using feature(s) ['Major_Axis_Length', 'Perimeter'] accuracy is 89.2%
	Using feature(s) ['Major_Axis_Length', 'Minor_Axis_Length'] accuracy is 89.0%
	Using feature(s) ['Major_Axis_Length', 'Eccentricity'] accuracy is 88.5%
	Using feature(s) ['Major_Axis_Length', 'Convex_Area'] accuracy is 88.5%
	Using feature(s) ['Major_Axis_Length', 'Extent'] accuracy is 88.9%

Feature set ['Major_Axis_Length', 'Perimeter'] was best, accuracy is 89.2%

	Using feature(s) ['Major_Axis_Length', 'Perim

(['Major_Axis_Length',
  'Perimeter',
  'Convex_Area',
  'Minor_Axis_Length',
  'Area'],
 0.8989501312335958)

### Performing backward elimination on the dataframe

In [38]:
backward_elimination(df)

Beginning search.

	Using feature(s) ['Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Convex_Area', 'Extent'] accuracy is 88.6%
	Using feature(s) ['Area', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Convex_Area', 'Extent'] accuracy is 88.5%
	Using feature(s) ['Area', 'Perimeter', 'Minor_Axis_Length', 'Eccentricity', 'Convex_Area', 'Extent'] accuracy is 88.6%
	Using feature(s) ['Area', 'Perimeter', 'Major_Axis_Length', 'Eccentricity', 'Convex_Area', 'Extent'] accuracy is 88.6%
	Using feature(s) ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Convex_Area', 'Extent'] accuracy is 88.7%
	Using feature(s) ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Extent'] accuracy is 88.6%
	Using feature(s) ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Convex_Area'] accuracy is 89.5%

Feature set ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Conv

(['Area',
  'Perimeter',
  'Major_Axis_Length',
  'Minor_Axis_Length',
  'Convex_Area'],
 0.8989501312335958)